# **0. Import Librerie**

In [1]:
!pip install scikit-learn==1.6.0 -q
import sklearn
print(sklearn.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 17.5 MB/s eta 0:00:00
1.6.0


In [2]:
import pandas as pd
import numpy as np
import urllib.request
import joblib
import plotly
import plotly.graph_objects as go

In [3]:
!pip install dash -q # Added -q for quiet installation
!pip install dash-bootstrap-components -q # Added -q for quiet installation

import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output, State

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 3.8 MB/s eta 0:00:00


# **1. Dashboard di predizione: GB.c. e ANN.r.**

Prima di utilizzare la dashboard per la predizione, assicurati di eseguire il Capitolo 0 per l'importazione delle librerie e il Capitolo 1 per la preparazione necessaria della dashboard.


*   GB.c. = Modello Gradient Boosting completo
*   ANN.r. = Modelo Artificial Neural Network ridotto



## 1.1. Dashboard modello Gradient Boosting Completo (GB.c.), *Acc. 0.67*

In [4]:
# @title 1.1.1. Modelli e parametri da caricare per GB.c.

'''
drive_path = '/content/drive/MyDrive/Tesi/Machine Learning/Saved Model/Gradient Boosting/Dash/'
accuracy_gb = joblib.load(drive_path + 'accuracy_gb.pkl')
scaler = joblib.load(drive_path + 'scaler.joblib')
poly = joblib.load(drive_path + 'poly.joblib')
selector = joblib.load(drive_path + 'selector.joblib')
loaded_model = joblib.load(drive_path + 'gradient_boosting_model.joblib')
'''

base_url = 'https://github.com/FedeGambe/Master_s_thesis_Data_science/raw/f94caa0c33c4716d7ebd6217502475a7f63bbdf3/Apprendimento_Automatico_analisi_predittive/Salvataggio_Modelli/Dash/Gradient_Boosting_Completo/'

files = {'accuracy_gb.pkl': 'accuracy_gb',
         'scaler.joblib': 'scaler',
         'poly.joblib': 'poly',
         'selector.joblib': 'selector',
         'gradient_boosting_model.joblib': 'loaded_model'}

for file_name, variable_name in files.items():  # scarica e carica ciascun file
    file_url = base_url + file_name             # crea link x ogni file
    file_path = f'/content/{file_name}'         # crea percorso in /content/ x ogni file

    print(f"Scaricando {file_name}...")     # scarica il file
    urllib.request.urlretrieve(file_url, file_path)

    globals()[variable_name] = joblib.load(file_path)     # carica il modello
    print(f"{file_name} caricato come {variable_name}!")

print("\nModelli caricati:") # Verifica sia stato caricato correttamente
print(f"accuracy_gb: {type(accuracy_gb)}")
print(f"scaler: {type(scaler)}")
print(f"poly: {type(poly)}")
print(f"selector: {type(selector)}")
print(f"loaded_model: {type(loaded_model)}")


Scaricando accuracy_gb.pkl...
accuracy_gb.pkl caricato come accuracy_gb!
Scaricando scaler.joblib...
scaler.joblib caricato come scaler!
Scaricando poly.joblib...
poly.joblib caricato come poly!
Scaricando selector.joblib...
selector.joblib caricato come selector!
Scaricando gradient_boosting_model.joblib...
gradient_boosting_model.joblib caricato come loaded_model!

Modelli caricati:
accuracy_gb: <class 'float'>
scaler: <class 'sklearn.preprocessing._data.StandardScaler'>
poly: <class 'sklearn.preprocessing._polynomial.PolynomialFeatures'>
selector: <class 'sklearn.feature_selection._univariate_selection.SelectKBest'>
loaded_model: <class 'sklearn.ensemble._gb.GradientBoostingClassifier'>


In [7]:
# @title 1.1.2. **Dashboard GB.c.**
def preprocess_data(data):
    scaled_data = scaler.transform(data)
    poly_data = poly.transform(data)
    selected_data = selector.transform(poly_data)
    return selected_data

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    html.H1("Dashboard di Predizione: modello GB", className="text-center mt-4 mb-4"),

    dbc.Row([#VARIABILI DEMOGRAFICHE
        dbc.Col([html.Label("Genere"),dcc.Dropdown(id='genere',options=[{'label': 'Maschio', 'value': 0},{'label': 'Femmina', 'value': 1}],value=0, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("Classe d'età"),dcc.Dropdown(id='classe_eta',options=[
                             {'label': '<25', 'value': 1},
                             {'label': '25-34', 'value': 2},
                             {'label': '35-44', 'value': 3},
                             {'label': '45-54', 'value': 4},
                             {'label': '54-64', 'value': 5},
                             {'label': '65-74', 'value': 6},
                             {'label': '75-79', 'value': 7},
                             {'label': '>80', 'value': 8}
                         ],value=1, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("Reddito familiare"),dcc.Dropdown(id='reddito',options=[
                             {'label': '<50,000', 'value': 50000},
                             {'label': '50,000 - 125,000', 'value': 125000},
                             {'label': '125,001 - 175,000', 'value': 175000},
                             {'label': '175,001 - 275,000', 'value': 275000},
                             {'label': '275,001- 500,000', 'value': 500000}
                         ], value=50000, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("Livello di istruzione"),dcc.Dropdown(id='istruzione',options=[
                             {'label': 'Licenza Media', 'value': 1},
                             {'label': 'Diploma', 'value': 2},
                             {'label': 'Laurea Triennale', 'value': 3},
                             {'label': 'Laurea Magistrale o Dottorato', 'value': 4},
                         ], value=1, className='form-control'),], width=6, className='mb-3'),]),

    dbc.Row([#VARIABILI ABITATIVE
        dbc.Col([html.Label("Numero di auto in famiglia"),dcc.Input(id='num_auto', type='number', value=1.0, min=1, max=6, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("Numero di persone in famiglia"),dcc.Input(id='num_pers', type='number', value=1.0, min=1, max=13, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("Casa di proprietà"),dcc.Dropdown(id='casa_prop',options=[{'label': 'No', 'value': 0}, {'label': 'Si', 'value': 1}],value=0, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("Casa Indipendente"),dcc.Dropdown(id='casa_ind',options=[{'label': 'No', 'value': 0}, {'label': 'Si', 'value': 1}],value=0, className='form-control'),], width=6, className='mb-3'),]),

#SENSIBILITà AMBIENTALE
    dbc.Row([dbc.Col([html.Label("Importanza di ridurre le emissioni di gas serra"),dcc.Slider(id='rid_emissioni', min=-3, max=3, step=0.1, value=0, marks={i: str(i) for i in range(-3,4)}, className='form-control'),], width=12, className='mb-3'),]),

    dbc.Row([#TIPOLOGIA DI AUTO
        dbc.Col([html.Label("Tipologia di auto precedente: ICE"),dcc.Dropdown(id='ice',options=[{'label': 'No', 'value': 0},{'label': 'Sì', 'value': 1}],value=0, className='form-control'),], width=4, className='mb-3'),
        dbc.Col([html.Label("Tipologia di auto precedente: BEV"),dcc.Dropdown(id='bev',options=[{'label': 'No', 'value': 0},{'label': 'Sì', 'value': 1}],value=0, className='form-control'),], width=4, className='mb-3'),
        dbc.Col([html.Label("Tipologia di auto precedente: HEV"),dcc.Dropdown(id='hev',options=[{'label': 'No', 'value': 0},{'label': 'Sì', 'value': 1}],value=0, className='form-control'),], width=4, className='mb-3'),
        dbc.Col([html.Label("Tipologia di auto precedente: PHEV"),dcc.Dropdown(id='phev',options=[{'label': 'No', 'value': 0},{'label': 'Sì', 'value': 1}],value=0, className='form-control'),], width=4, className='mb-3'),
        dbc.Col([html.Label("Tipologia di auto precedente: GNC"),dcc.Dropdown(id='gnc',options=[{'label': 'No', 'value': 0}, {'label': 'Sì', 'value': 1}],value=0, className='form-control'),], width=4, className='mb-3'),]),

    dbc.Row([#VARIABILI MOBILITà
        dbc.Col([html.Label("Numero di viaggi superiori a 200 miglia negli ultimi 12 mesi"),dcc.Input(id='num_viaggi', type='number', value=0.0, min=0, max=65, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("Viaggio più lungo negli ultimi 12 mesi"),dcc.Input(id='viaggio_lungo', type='number', value=0,min=0, max=4800, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("Distanza casa-lavoro"),dcc.Input(id='distanza_lavoro', type='number', value=0.0, min=0, max=300, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("VMT annuo"),dcc.Input(id='vmt_annuo', type='number', value=0.0, min=0, max=100000, className='form-control'),], width=6, className='mb-3')]),

    #html.Div(id='debug-output', className='mt-4 text-center'),
    #html.Div([dbc.Button('Genera Predizione', id='predict-button', n_clicks=0, color='primary', className='mt-4'),html.Div(id='prediction-output', className='mt-4 text-center'),dcc.Graph(id='prediction-graph', config={'displayModeBar': False}),])
    html.Div([
        dbc.Row([dbc.Col(dbc.Button('Genera Predizione', id='predict-button', n_clicks=0, color='primary', className='mt-4'),width={'size': 6, 'offset': 3},className="text-center")]),
        html.Div(id='prediction-output', className='mt-4 text-center'),
        dcc.Graph(id='prediction-graph', config={'displayModeBar': False})]),
], fluid=True)

@app.callback(
    [Output('prediction-output', 'children'),
     Output('prediction-graph', 'figure')],
    [Input('predict-button', 'n_clicks')],
    [
        State('num_viaggi', 'value'),
        State('vmt_annuo', 'value'),
        State('reddito', 'value'),
        State('bev', 'value'),
        State('rid_emissioni', 'value'),
        State('classe_eta', 'value'),
        State('viaggio_lungo', 'value'),
        State('distanza_lavoro', 'value'),
        State('num_auto', 'value'),
        State('genere', 'value'),
        State('ice', 'value'),
        State('hev', 'value'),
        State('istruzione', 'value'),
        State('num_pers', 'value'),
        State('casa_prop', 'value'),
        State('casa_ind', 'value'),
        State('phev', 'value'),
        State('gnc', 'value')
    ]
)
def update_prediction(n_clicks, num_viaggi, vmt_annuo, reddito, bev, rid_emissioni, classe_eta, viaggio_lungo, distanza_lavoro, num_auto, genere, ice, hev, istruzione, num_pers, casa_prop, casa_ind, phev, gnc):
    if n_clicks > 0:
        try:
            new_data = np.array([[num_viaggi, vmt_annuo, reddito, bev, rid_emissioni, classe_eta, viaggio_lungo, distanza_lavoro, num_auto, genere, ice, hev, istruzione, num_pers, casa_prop, casa_ind, phev, gnc]]).reshape(1, -1)
            new_data_processed = preprocess_data(new_data)

            probability = loaded_model.predict_proba(new_data_processed)[0]             # Calcola la probabilità con il modello

            predicted_class = np.argmax(probability)             # Determina la classe con la probabilità più alta
            predicted_probability = probability[predicted_class]

            classes = ['Classe 0', 'Classe 1']             # Crea il grafico delle probabilità
            figure = go.Figure(data=[go.Bar(x=classes, y=probability, marker_color=['blue', 'orange'])])
            figure.update_layout(title='Predizione delle Classi', xaxis_title='Classe', yaxis_title='Probabilità')

            return f'Accuratezza predizione: {(accuracy_gb * 100):.2f} %, Classe Predetta: {predicted_class} con una probabilità del {predicted_probability * 100:.2f}%', figure
        except Exception as e:
            return f"Si è verificato un errore durante la predizione: {e}", go.Figure()
    return 'Inserisci i valori delle variabili e premi il pulsante per generare la predizione.', go.Figure()

#####___________________Mutua Esclusività
@app.callback(
    [Output('ice', 'value'), Output('bev', 'value'), Output('hev', 'value')],
    [Input('ice', 'value'), Input('bev', 'value'), Input('hev', 'value')],
    prevent_initial_call=True
)
def handle_exclusivity(ice, bev, hev):
    ctx = dash.callback_context
    if not ctx.triggered:
        return ice, bev, hev
    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]
    if triggered_id == 'ice' and ice == 1:
        return ice, 0, 0
    elif triggered_id == 'bev' and bev == 1:
        return 0, bev, 0
    elif triggered_id == 'hev' and hev == 1:
        return 0, 0, hev
    return ice, bev, hev

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

## 1.2. Dashboard modello ANN Ridotto (ANN.r.), *Acc. 0.65*

In [ ]:
# @title Modelli e parametri da caricare per ANN.r.
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LeakyReLU

drive_path = '/content/drive/MyDrive/Tesi/Machine Learning/Saved Model/Tensorflow/Model_Reduced_FE/'
scaler = joblib.load(drive_path + 'scaler.pkl')
poly = joblib.load(drive_path + 'poly.pkl')
selector = joblib.load(drive_path + 'selector.pkl')
drive_path1 = '/content/drive/MyDrive/Tesi/Machine Learning/Saved Model/Tensorflow/'
model = load_model(drive_path1 + 'model2_rid_full_model.keras', custom_objects={'LeakyReLU': LeakyReLU(negative_slope=0.3)})

In [ ]:
# @title Dashboard ANN.r.
# Funzione per la pre-elaborazione dei dati
def preprocess_data(data):
    scaled_data = scaler.transform(data)
    poly_data = poly.transform(scaled_data)
    selected_data = selector.transform(poly_data)
    return selected_data

# Inizializza l'app Dash
app_ann = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout dell'app
app_ann.layout = dbc.Container([
    html.H1("Dashboard di Predizione: modello ANN ridotto", className="text-center mt-4 mb-4"),
    dbc.Row([
        dbc.Col([html.Label("Genere"),dcc.Dropdown(id='genere',
                options=[
                    {'label': 'Maschio', 'value': 0},
                    {'label': 'Femmina', 'value': 1}
                ],value=0, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("Classe d'età"),dcc.Dropdown(id='classe_eta',
                options=[
                    {'label': '>25', 'value': 1},
                    {'label': '25-34', 'value': 2},
                    {'label': '35-44', 'value': 3},
                    {'label': '45-54', 'value': 4},
                    {'label': '54-64', 'value': 5},
                    {'label': '65-74', 'value': 6},
                    {'label': '75-79', 'value': 7},
                    {'label': '>80', 'value': 8}
                ],value=1, className='form-control'),], width=6, className='mb-3'),]),
    dbc.Row([
        dbc.Col([html.Label("Reddito familiare"),dcc.Dropdown(id='reddito',
                options=[
                    {'label': '>50,000', 'value': 50000},
                    {'label': '50,000 - 125,000', 'value': 125000},
                    {'label': '125,001 - 175,000', 'value': 175000},
                    {'label': '175,001 - 275,000', 'value': 275000},
                    {'label': '275,001- 500,000', 'value': 500000}
                ],value=50000, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("Numero di auto in famiglia"),dcc.Input(id='num_auto', type='number', value=1.0, className='form-control'),], width=6, className='mb-3'),]),
    dbc.Row([
        dbc.Col([html.Label("Tipologia di auto precedente: ICE"),dcc.Dropdown(id='ice',
                options=[
                    {'label': 'No', 'value': 0},
                    {'label': 'Sì', 'value': 1}
                ],value=0, className='form-control'),], width=4, className='mb-3'),
        dbc.Col([html.Label("Tipologia di auto precedente: BEV"),dcc.Dropdown(id='bev',
                options=[
                    {'label': 'No', 'value': 0},
                    {'label': 'Sì', 'value': 1}
                ],value=0, className='form-control'),], width=4, className='mb-3'),
        dbc.Col([html.Label("Tipologia di auto precedente: HEV"),dcc.Dropdown(id='hev',
                options=[
                    {'label': 'No', 'value': 0},
                    {'label': 'Sì', 'value': 1}
                ],value=0, className='form-control'),], width=4, className='mb-3'), ]),

#SENSIBILITà AMBIENTALE
    dbc.Row([dbc.Col([html.Label("Importanza di ridurre le emissioni di gas serra"),dcc.Slider(id='rid_emissioni', min=-3, max=3, step=0.1, value=0.0, marks={i: str(i) for i in range(-3,4)}, className='form-control'),], width=12, className='mb-3'),]),
    dbc.Row([
        dbc.Col([html.Label("Numero di viaggi superiori a 200 miglia negli ultimi 12 mesi"),dcc.Input(id='num_viaggi', type='number', value=0.0, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("Viaggio più lungo negli ultimi 12 mesi"),dcc.Input(id='viaggio_lungo', type='number', value=0.210, className='form-control'),], width=6, className='mb-3'),]),
    dbc.Row([
        dbc.Col([html.Label("Distanza casa-lavoro"),dcc.Input(id='distanza_lavoro', type='number', value=0.0, className='form-control'),], width=6, className='mb-3'),
        dbc.Col([html.Label("VMT annuo"),dcc.Input(id='vmt_annuo', type='number', value=0.0, className='form-control'),], width=6, className='mb-3')]),
    html.Div([
        dbc.Button('Genera Predizione', id='predict-button', n_clicks=0, color='primary', className='mt-4'),
        html.Div(id='prediction-output', className='mt-4 text-center'),
        dcc.Graph(id='prediction-graph', config={'displayModeBar': False}),])
], fluid=True)

# Callback per generare la predizione e aggiornare il grafico
@app_ann.callback(
    [Output('prediction-output', 'children'),
     Output('prediction-graph', 'figure')],
    [Input('predict-button', 'n_clicks')],
    [State('num_viaggi', 'value'),
     State('vmt_annuo', 'value'),
     State('reddito', 'value'),
     State('bev', 'value'),
     State('rid_emissioni', 'value'),
     State('classe_eta', 'value'),
     State('viaggio_lungo', 'value'),
     State('distanza_lavoro', 'value'),
     State('num_auto', 'value'),
     State('genere', 'value'),
     State('ice', 'value'),
     State('hev', 'value')])

def update_prediction(n_clicks, num_viaggi, vmt_annuo, reddito, bev, rid_emissioni, classe_eta, viaggio_lungo, distanza_lavoro, num_auto, genere, ice, hev):
    if n_clicks > 0:
        try:
            new_data = np.array([[num_viaggi, vmt_annuo, reddito, bev, rid_emissioni, classe_eta, viaggio_lungo, distanza_lavoro, num_auto, genere, ice, hev]])
            new_data_processed = preprocess_data(new_data)
            prediction = model.predict(new_data_processed)[0][0]

            # Creare il grafico delle predizioni
            classes = ['Classe 0', 'Classe 1']
            values = [1 - prediction, prediction]
            figure = go.Figure(data=[
                go.Bar(x=classes, y=values, marker_color=['blue', 'orange'])
            ])
            figure.update_layout(title='Predizione delle Classi', xaxis_title='Classe', yaxis_title='Probabilità')

            return f'Predizione: {prediction}', figure
        except Exception as e:
            return f"Si è verificato un errore durante la predizione: {e}", go.Figure()
    return 'Inserisci i valori delle variabili e premi il pulsante per generare la predizione.', go.Figure()


# Callback per gestire la mutua esclusività delle variabili "Tipologia di auto precedente"
@app_ann.callback(
    [Output('ice', 'value'), Output('bev', 'value'), Output('hev', 'value')],
    [Input('ice', 'value'), Input('bev', 'value'), Input('hev', 'value')],
    prevent_initial_call=True
)
def handle_exclusivity(ice, bev, hev):
    ctx = dash.callback_context

    if not ctx.triggered:
        return ice, bev, hev

    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if triggered_id == 'ice' and ice == 1:
        return ice, 0, 0
    elif triggered_id == 'bev' and bev == 1:
        return 0, bev, 0
    elif triggered_id == 'hev' and hev == 1:
        return 0, 0, hev
    return ice, bev, hev

# Esegui l'app Dash
if __name__ == '__main__':
    app_ann.run_server(debug=True)

<IPython.core.display.Javascript object>

# **2. Preparazione dataset per ventuale verifica** (non avviare)

##1.1. Importazione Dati

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict, StratifiedKFold, KFold, cross_validate, learning_curve
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, auc, precision_recall_curve, f1_score, precision_score, recall_score

In [ ]:
data= pd.read_csv("/content/drive/MyDrive/Tesi/dataset_originale.csv", sep=",")

for col in data.columns:
    print(f'"{col}"')

"Genere"
"Classe d'età"
"Reddito familiare"
"Classe Reddito Familiare"
"Livello di istruzione"
"Casa di proprietà"
"Casa Indipendente"
"Numero persone in famiglia"
"Numero di auto in famiglia"
"Importanza di ridurre le emissioni di gas serra"
"Tipologia di auto precedente"
"Auto attuale"
"Tipologia di auto attuale"
"BEV dummy"
"Viaggio più lungo negli ultimi 12 mesi"
"Numero di viaggi superiori a 200 miglia negli ultimi 12 mesi"
"Distanza casa-lavoro"
"VMT annuo"


In [ ]:
data = data.drop(["Auto attuale", "Tipologia di auto attuale"
                  ], axis=1)

In [ ]:
for col in data.select_dtypes(include=['object']).columns:
    unique_col = data[col].unique()
    print(col, unique_col)

Genere ['Maschio' 'Femmina']
Classe d'età ['65-74' '55-64' '75-79' '45-54' '35-44' '<25' '25-34' '>80']
Classe Reddito Familiare ['Molto alta' 'Media' 'Alta' 'Estremamente alta' 'Bassa']
Livello di istruzione ['Laurea 1L' 'Laurea 2L o Dottorato' 'Diploma o Qualifica professionale'
 'Licenza Media']
Casa di proprietà ['Si' 'No']
Casa Indipendente ['Si' 'No']
Tipologia di auto precedente ['PHEV' 'HEV' 'ICE' 'BEV' 'GNC']


In [ ]:
Genere_mapping = {'Maschio':1, 'Femmina':0}
Eta_mapping = {'<25': 1, '25-34':2, '35-44':3, '45-54':4, '55-64':5, '65-74':6, '75-79':7, '>80':8}
Isruzione_mapping = {'Licenza Media':1, 'Diploma o Qualifica professionale':2, 'Laurea 1L':3, 'Laurea 2L o Dottorato':4}
Reddito_mapping = {'Bassa':1, 'Media':2, 'Alta':3, 'Molto alta':4, 'Estremamente alta':5}
casapro_mapping = {'Si':1, 'No':0}
casaind_mapping = {'Si':1, 'No':0}
#autoprec_mapping = {'PHEV':1, 'BEV':2, 'HEV':3, 'ICE':4, 'GNC':5}

data['Genere'] = data['Genere'].map(Genere_mapping).values
data["Classe d'età"] = data["Classe d'età"].map(Eta_mapping).values
data['Livello di istruzione'] = data['Livello di istruzione'].map(Isruzione_mapping).values
data['Classe Reddito Familiare'] = data['Classe Reddito Familiare'].map(Reddito_mapping).values
data['Casa di proprietà'] = data['Casa di proprietà'].map(casapro_mapping).values
data['Casa Indipendente'] = data['Casa Indipendente'].map(casaind_mapping).values

data['Tipologia di auto precedente: PHEV'] = (data['Tipologia di auto precedente'] == 'PHEV').astype(int)
data['Tipologia di auto precedente: BEV'] = (data['Tipologia di auto precedente'] == 'BEV').astype(int)
data['Tipologia di auto precedente: HEV'] = (data['Tipologia di auto precedente'] == 'HEV').astype(int)
data['Tipologia di auto precedente: ICE'] = (data['Tipologia di auto precedente'] == 'ICE').astype(int)
data['Tipologia di auto precedente: GNC'] = (data['Tipologia di auto precedente'] == 'GNC').astype(int)

data = data.drop('Tipologia di auto precedente', axis=1)
data = data.drop("Classe Reddito Familiare", axis=1)

In [ ]:
y = data['BEV dummy']
X = data.drop(columns=['BEV dummy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=343)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
X_train.shape

(8550, 18)

In [ ]:
X_test.shape

(2138, 18)

## 1.2. Preparazione modello Gradient Boosting

In [ ]:
###Preprocessing e Feature Engineering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

selector = SelectKBest(score_func=f_classif, k='all')
X_selected = selector.fit_transform(X_poly, y)
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(learning_rate=0.05, max_depth=3, min_samples_split=5, n_estimators=200, subsample=0.7)
gb.fit(X_train, y_train)
predictions = gb.predict(X_test)
accuracy_gb = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy_gb:.2f}')

Accuracy: 0.67
